# pymysql
- 파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
- Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따름
- https://github.com/PyMySQL/PyMySQL/
- https://pymysql.readthedocs.io/en/latest/

# 설치
- 조건
    - python version 3.6 이상
    - mysql version 5.6 이상
- 설치
    - `pip install PyMySQL`
    - `conda install -c conda-forge pymysql`

# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip", 
                                     port="port번호", 
                                     user="계정명", 
                                     password="비밀번호", 
                                     db="연결할데이터베이스이름", 
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫음
    - with문을 이용 가능
    ```python
    cursor.close()
    connection.close()
    ```

# 예제

## 테이블 생성
- pymysql을 이용해서 DML(insert, update, delete)문을 실행 (DDL은 거의 미사용)

In [1]:
# import
import pymysql

# DB연결 -connect() 연결 실패 시 Exception 발생
connection = pymysql.connect(host='127.0.0.1',
                             port=3306,
                             user='scott',
                             password='tiger',
                             db='testdb',
                             charset='utf8')
print(type(connection))

<class 'pymysql.connections.Connection'>


In [2]:
# cursor를 connection으로 부터 얻어옴 - cursor : sql문 실행을 관리하는 객체
cursor = connection.cursor()

In [3]:
# sql문 실행 - 실행 실패(sql문 오작성, 제약조건 문제) 시 Exception 발생
# sql문은 string정의 execute() 메소드를 이용하여 DB에 전송
sql = '''
create table test_user(
    id int auto_increment primary key,
    name varchar(30) not null,
    email varchar(100) not null unique,
    tall decimal(5, 2),
    birthday date
)
'''

cursor.execute(sql)

0

In [4]:
# 연결 닫기 - cursor, connection close
cursor.close()
connection.close()

## DML
### insert

In [5]:
import pymysql

sql = "insert into test_user (name, email, tall, birthday) values ('홍길동', 'a@a.com', 182.23, '2000-02-03')"
try:
    # connect
    connection = pymysql.connect(host='127.0.0.1', port=3306, user='scott', password = 'tiger', db = 'testdb', charset='utf8')
    # cursor 생성
    cursor = connection.cursor()
    # insert 실행
    cnt = cursor.execute(sql)
    connection.commit # insert/delete/update 문 실행 후 commit 처리를 해야함
    print(f'{cnt}행이 insert 되었습니다.')

except Exception as e:
    if connection:
        connection.rollback()
    print(e)
finally:
    # 연결 닫기
    if cursor: # if cursor != None
        cursor.close()
    if connection: # if connection != None
        connection.close()

1행이 insert 되었습니다.


In [6]:
# with 문
# with 연결함수 as 변수
sql = "insert into test_user (name, email, tall, birthday) values ('강감찬', 'b@b.com', 188.00, '2001-10-13')"
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(sql)
        connection.commit()
        print(f'{cnt}행이 insert 되었습니다.')

1행이 insert 되었습니다.


In [7]:
# 함수를 이용한 삽입
def insert_user(name, email, tall, birthday):
    sql = f"insert into test_user (name, email, tall, birthday) values ('{name}', '{email}', {tall}, '{birthday}')"
    with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql)
            connection.commit()

insert_user('이순신', 'c@c.com', 175.00, '1995-10-22')

### Parameterized Query
- Parameterized Query
    - SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용 후 execute()에서 placeholder에 넣을 값을 tuple로 제공
    - query문을 쉽게 작성할 수 있는 장점

In [8]:
# datetime 모듈 : 날짜(date), 시간(time), 날짜시간(datetime) 등 날짜와 시간을 다루는 모듈, DB의 날짜/시간과 연동
from datetime import date

sql = "insert into test_user (name, email, tall, birthday) values (%s, %s, %s, %s)"
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(sql, ('유관순', 'd@d.com', 175.0, '2000-01-02'))
        cnt2 = cursor.execute(sql, ('김유신', 'e@e.com', 179.5, date(1995, 2, 3)))
        connection.commit()
        print(f'{cnt}행이 insert 되었습니다.')
        print(f'{cnt2}행이 insert 되었습니다.')

1행이 insert 되었습니다.
1행이 insert 되었습니다.


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용

In [9]:
names = ['이름1', '이름2', '이름3']
emails = ['f@f.com', 'g@g.com' ,'h@h.com']
talls = [177.7, 163.9, 183.6]
birthdays = ['2002-06-24', date.today(), '1999-05-05']

In [10]:
sql = "insert into test_user (name, email, tall, birthday) values (%s, %s, %s, %s)"
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        for data in zip (names, emails, talls, birthdays):
            cursor.execute(sql, data)
            
        connection.commit()

#### executemany() 사용
- insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert

In [11]:
datas = [
    ['name1', 'i@i.com', 171.1, '1991-01-01'],
    ['name2', 'j@j.com', 172.2, '1992-02-02'],
    ['name3', 'k@k.com', 173.3, '1993-03-03'],
]
sql = "insert into test_user (name, email, tall, birthday) values (%s, %s, %s, %s)"
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        cursor.executemany(sql, datas)
        connection.commit()

### update/delete
- 코딩 절차는 insert 와 동일

In [12]:
update_sql = "update test_user set tall = tall + 10, birthday = '2000-01-01'"
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(update_sql)
        connection.commit()
        print(f'{cnt}행 update 완료')

10행 update 완료


In [13]:
def update_user_by_id(id, name, email, tall, birthday):
    update_sql = 'update test_user set name=%s, email=%s, tall=%s, birthday=%s where id=%s'
    with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(update_sql, (name, email, tall, birthday, id))
            connection.commit()
            return cnt

In [14]:
update_user_by_id(2, 'Hong', 'l@l.com', 188.88, '1990-10-10')

1

In [15]:
# delete
delete_sql = "delete from test_user where id = %s"
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(delete_sql, (2,))
        print(cnt)
        connection.commit()

1


In [16]:
def delete_user_by_id(id):
    delete_sql = "delete from test_user where id = %s"
    with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(delete_sql, (id,))
            connection.commit()
            return cnt

In [17]:
delete_user_by_id(4)

1

## select (DQL)
- 조회결과 조회
    - cusor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받음
- fetch메소드
    - **fetchall()**
        - 조회한 모든 행을을 반환
    - **fetchmany(size=개수)**
        - 지정한 size개수 만큼 반환
    - **fetchone()**
        - 조회결과 중 첫번째 행만 반환
        - 주로 pk 동등 조건으로 조회한 경우 사용

### fetchall()

In [18]:
import pymysql

sql = 'select * from test_user'
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        # select 문 실행
        cursor.execute(sql)
        # select 결과 조회(cursor.fetchall() 메소드 이용)
        result = cursor.fetchall()

In [19]:
print(type(result), len(result))
print(type(result[0]))
result

<class 'tuple'> 8
<class 'tuple'>


((3, '이순신', 'c@c.com', Decimal('185.00'), datetime.date(2000, 1, 1)),
 (5, '김유신', 'e@e.com', Decimal('189.50'), datetime.date(2000, 1, 1)),
 (6, '이름1', 'f@f.com', Decimal('187.70'), datetime.date(2000, 1, 1)),
 (7, '이름2', 'g@g.com', Decimal('173.90'), datetime.date(2000, 1, 1)),
 (8, '이름3', 'h@h.com', Decimal('193.60'), datetime.date(2000, 1, 1)),
 (9, 'name1', 'i@i.com', Decimal('181.10'), datetime.date(2000, 1, 1)),
 (10, 'name2', 'j@j.com', Decimal('182.20'), datetime.date(2000, 1, 1)),
 (11, 'name3', 'k@k.com', Decimal('183.30'), datetime.date(2000, 1, 1)))

In [20]:
# 커서 클래스 변경
sql = 'select * from test_user'
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8', cursorclass = pymysql.cursors.DictCursor) as connection: # pymysql.cursor.Dictcursor :  조회결과를 dictionary로 반환하는 커서
    with connection.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()

In [21]:
print(type(result), len(result))
print(type(result[0]))
print(result)
print(result[0]['id'], result[0]['email'])

<class 'list'> 8
<class 'dict'>
[{'id': 3, 'name': '이순신', 'email': 'c@c.com', 'tall': Decimal('185.00'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 5, 'name': '김유신', 'email': 'e@e.com', 'tall': Decimal('189.50'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 6, 'name': '이름1', 'email': 'f@f.com', 'tall': Decimal('187.70'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 7, 'name': '이름2', 'email': 'g@g.com', 'tall': Decimal('173.90'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 8, 'name': '이름3', 'email': 'h@h.com', 'tall': Decimal('193.60'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 9, 'name': 'name1', 'email': 'i@i.com', 'tall': Decimal('181.10'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 10, 'name': 'name2', 'email': 'j@j.com', 'tall': Decimal('182.20'), 'birthday': datetime.date(2000, 1, 1)}, {'id': 11, 'name': 'name3', 'email': 'k@k.com', 'tall': Decimal('183.30'), 'birthday': datetime.date(2000, 1, 1)}]
3 c@c.com


In [22]:
sql = 'select id, name from test_user'
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8', 
                     ) as connection:   # 조회결과를 dictionary로 반환하는 커서
    with connection.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
print(result)
print("2번 행의 이름:", result[1][1])

((3, '이순신'), (5, '김유신'), (6, '이름1'), (7, '이름2'), (8, '이름3'), (9, 'name1'), (10, 'name2'), (11, 'name3'))
2번 행의 이름: 김유신


### fetchone()

In [23]:
sql = 'select * from test_user where id = %s' # pk로 조회 결과는 0 또는 1 행
with pymysql.connect(host = '127.0.0.1', port = 3306, user = 'scott', password = 'tiger', db = 'testdb', charset = 'utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql, (10, ))
        result = cursor.fetchall()
print(result)
if result:
    print(len(result))

((10, 'name2', 'j@j.com', Decimal('182.20'), datetime.date(2000, 1, 1)),)
1


In [24]:
sql = 'select * from test_user' 
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)    
        result = cursor.fetchone()  # 여러행이 조회된 경우에서 한행만(첫번째행) 반환

In [25]:
print(result)

(3, '이순신', 'c@c.com', Decimal('185.00'), datetime.date(2000, 1, 1))


### fetchmany()

In [26]:
sql = "select * from test_user"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchmany(size=3)

In [27]:
print(len(result))
print(result)

3
((3, '이순신', 'c@c.com', Decimal('185.00'), datetime.date(2000, 1, 1)), (5, '김유신', 'e@e.com', Decimal('189.50'), datetime.date(2000, 1, 1)), (6, '이름1', 'f@f.com', Decimal('187.70'), datetime.date(2000, 1, 1)))


### curor 는 iterable 타입
- for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회 가능

In [28]:
sql = "select * from test_user"
with pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', db='testdb', charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.execute(sql)
        for data in cursor:  #한번 반복시 마다 조회결과를 한행씩 반환
            print(data)

(3, '이순신', 'c@c.com', Decimal('185.00'), datetime.date(2000, 1, 1))
(5, '김유신', 'e@e.com', Decimal('189.50'), datetime.date(2000, 1, 1))
(6, '이름1', 'f@f.com', Decimal('187.70'), datetime.date(2000, 1, 1))
(7, '이름2', 'g@g.com', Decimal('173.90'), datetime.date(2000, 1, 1))
(8, '이름3', 'h@h.com', Decimal('193.60'), datetime.date(2000, 1, 1))
(9, 'name1', 'i@i.com', Decimal('181.10'), datetime.date(2000, 1, 1))
(10, 'name2', 'j@j.com', Decimal('182.20'), datetime.date(2000, 1, 1))
(11, 'name3', 'k@k.com', Decimal('183.30'), datetime.date(2000, 1, 1))
